In [1]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices, create_submission, write_submission
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeTwoModelsByScores, MergeThreeModelsByScores

In [2]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()
URM_stacked = combine_matrices(ICM=ICM, URM=URM)

In [3]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_stacked, train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage=0.85)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13923 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13905 ( 0.6%) Users that have less than 1 test interactions


In [4]:
stacked_train = combine_matrices(ICM=ICM, URM= URM_train)

In [5]:
### RP3Beta Recommender
RP3Beta_recommender = RP3betaRecommender(URM_train=URM_train)
RP3Beta_recommender.fit(topK=181, alpha=0.5709402717259106, beta=0.3566066452521787, implicit=True, normalize_similarity=True)

RP3betaRecommender: URM Detected 4 ( 0.0%) users with no interactions.


In [6]:
##SLIMELASTICNET Recommender
SLIME_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_stacked)
SLIME_recommender.fit(topK=420, l1_ratio=0.06061216785568925, alpha=0.00554982938879437, workers = 10)

100%|████████████████████████████████████▉| 18032/18059 [12:42<00:00, 51.22it/s]

In [7]:
similarity_slime = SLIME_recommender.W_sparse
similarity_rp3 = RP3Beta_recommender.W_sparse

In [8]:

similarity_hybrid = ItemKNNSimilarityHybridRecommender(URM_train = URM_stacked,
                                                      Similarity_1 = similarity_rp3,
                                                      Similarity_2 = similarity_slime)
similarity_hybrid.fit(topK=494, alpha=0.1597863887003789 )

In [9]:
## IALS Recommender
IALS_recommender = IALSRecommender(URM_train=URM_stacked)
IALS_recommender.fit(epochs=10, num_factors=42, alpha=0.7617528864750021, reg=8.926401306541349)

IALSRecommenderLinear: Epoch 1 of 10. Elapsed time 9.35 sec


100%|████████████████████████████████████▉| 18056/18059 [13:00<00:00, 51.22it/s]

IALSRecommenderLinear: Epoch 2 of 10. Elapsed time 19.21 sec
IALSRecommenderLinear: Epoch 3 of 10. Elapsed time 29.18 sec
IALSRecommenderLinear: Epoch 4 of 10. Elapsed time 38.26 sec
IALSRecommenderLinear: Epoch 5 of 10. Elapsed time 47.64 sec
IALSRecommenderLinear: Epoch 6 of 10. Elapsed time 57.82 sec
IALSRecommenderLinear: Epoch 7 of 10. Elapsed time 1.12 min
IALSRecommenderLinear: Epoch 8 of 10. Elapsed time 1.27 min
IALSRecommenderLinear: Epoch 9 of 10. Elapsed time 1.44 min
IALSRecommenderLinear: Epoch 10 of 10. Elapsed time 1.60 min
IALSRecommenderLinear: Terminating at epoch 10. Elapsed time 1.60 min


In [10]:
##UserKNNCF
UserKNNCF_recommender = UserKNNCFRecommender(URM_train = stacked_train)
UserKNNCF_recommender.fit(topK=450, shrink=5.0, similarity="tanimoto")

UserKNNCFRecommender: URM Detected 4 ( 0.0%) users with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 14318 (100.0%), 958.64 column/sec. Elapsed time 14.94 sec


In [11]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from skopt.space import Real
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

hyperparameters_range_dictionary = {
    "alpha": Real(0.0, 1.0),
    "beta": Real(0.0, 1.0)
}
recommenders = [similarity_hybrid, UserKNNCF_recommender]

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[stacked_train, recommenders],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class=MergeTwoModelsByScores,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 200
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=MergeTwoModelsByScores.RECOMMENDER_NAME,
                    save_model="best"
                    )




Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'alpha': 0.5984603383144845, 'beta': 0.9622179131246718}
MergeTwoModelsByScore: URM Detected 4 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13905 (100.0%) in 18.02 sec. Users per second: 772
SearchBayesianSkopt: New best config found. Config 0: {'alpha': 0.5984603383144845, 'beta': 0.9622179131246718} - results: PRECISION: 0.2199928, PRECISION_RECALL_MIN_DEN: 0.2291616, RECALL: 0.0707140, MAP: 0.1070709, MAP_MIN_DEN: 0.1120857, MRR: 0.4469637, NDCG: 0.2332891, F1: 0.1070259, HIT_RATE: 0.8911183, ARHR_ALL_HITS: 0.6882292, NOVELTY: 0.0054940, AVERAGE_POPULARITY: 0.5940313, DIVERSITY_MEAN_INTER_LIST: 0.8442103, DIVERSITY_HERFINDAHL: 0.9844150, COVERAGE_ITEM: 0.1776400, COVERAGE_ITEM_CORRECT: 0.0859959, COVERAGE_USER: 0.9943507, COVERAGE_USER_CORRECT: 0.8860841, DIVERSITY_GINI: 0.0100420, SHANNON_ENTROPY: 7.1493546, RATIO_DIVERSITY_HERFINDAHL: 0.9847784, RATIO_DIVERSITY_GI

100%|████████████████████████████████████▉| 18056/18059 [15:35<00:00, 19.31it/s]


EvaluatorHoldout: Processed 13923 (100.0%) in 17.67 sec. Users per second: 788
SearchBayesianSkopt: Config evaluated with evaluator_test. Config: {'alpha': 0.5379568712470829, 'beta': 0.8004552478406813} - results:
CUTOFF: 10 - PRECISION: 0.2539108, PRECISION_RECALL_MIN_DEN: 0.2616972, RECALL: 0.0698426, MAP: 0.1289195, MAP_MIN_DEN: 0.1335669, MRR: 0.4885662, NDCG: 0.2666682, F1: 0.1095512, HIT_RATE: 0.9248725, ARHR_ALL_HITS: 0.7880996, NOVELTY: 0.0054945, AVERAGE_POPULARITY: 0.5938519, DIVERSITY_MEAN_INTER_LIST: 0.8442975, DIVERSITY_HERFINDAHL: 0.9844237, COVERAGE_ITEM: 0.1786921, COVERAGE_ITEM_CORRECT: 0.0941359, COVERAGE_USER: 0.9956379, COVERAGE_USER_CORRECT: 0.9208381, DIVERSITY_GINI: 0.0101093, SHANNON_ENTROPY: 7.1536043, RATIO_DIVERSITY_HERFINDAHL: 0.9847871, RATIO_DIVERSITY_GINI: 0.0378440, RATIO_SHANNON_ENTROPY: 0.5732166, RATIO_AVERAGE_POPULARITY: 3.0520685, RATIO_NOVELTY: 0.0290171, 


SearchBayesianSkopt: Saving model in logs/MergeTwoModelsByScore

MergeTwoModelsByScore: Sa

EvaluatorHoldout: Processed 13905 (100.0%) in 17.95 sec. Users per second: 775
SearchBayesianSkopt: Config 6 is suboptimal. Config: {'alpha': 0.69492228184861, 'beta': 0.5958412136759025} - results: PRECISION: 0.2224380, PRECISION_RECALL_MIN_DEN: 0.2315866, RECALL: 0.0715310, MAP: 0.1081903, MAP_MIN_DEN: 0.1132133, MRR: 0.4481430, NDCG: 0.2353511, F1: 0.1082510, HIT_RATE: 0.8945703, ARHR_ALL_HITS: 0.6930727, NOVELTY: 0.0054914, AVERAGE_POPULARITY: 0.5937714, DIVERSITY_MEAN_INTER_LIST: 0.8445422, DIVERSITY_HERFINDAHL: 0.9844481, COVERAGE_ITEM: 0.1734315, COVERAGE_ITEM_CORRECT: 0.0866604, COVERAGE_USER: 0.9943507, COVERAGE_USER_CORRECT: 0.8895166, DIVERSITY_GINI: 0.0099444, SHANNON_ENTROPY: 7.1575602, RATIO_DIVERSITY_HERFINDAHL: 0.9848116, RATIO_DIVERSITY_GINI: 0.0372267, RATIO_SHANNON_ENTROPY: 0.5735335, RATIO_AVERAGE_POPULARITY: 3.0516549, RATIO_NOVELTY: 0.0290006, 

Iteration No: 7 ended. Evaluation done at random point.
Time taken: 18.1165
Function value obtained: -0.1082
Current min

EvaluatorHoldout: Processed 13905 (100.0%) in 17.62 sec. Users per second: 789
SearchBayesianSkopt: Config 13 is suboptimal. Config: {'alpha': 0.09714459554655522, 'beta': 0.699329251052277} - results: PRECISION: 0.2162819, PRECISION_RECALL_MIN_DEN: 0.2250344, RECALL: 0.0690009, MAP: 0.1053313, MAP_MIN_DEN: 0.1103234, MRR: 0.4442087, NDCG: 0.2300015, F1: 0.1046235, HIT_RATE: 0.8853650, ARHR_ALL_HITS: 0.6805111, NOVELTY: 0.0055036, AVERAGE_POPULARITY: 0.5925276, DIVERSITY_MEAN_INTER_LIST: 0.8455069, DIVERSITY_HERFINDAHL: 0.9845446, COVERAGE_ITEM: 0.1945290, COVERAGE_ITEM_CORRECT: 0.0858852, COVERAGE_USER: 0.9943507, COVERAGE_USER_CORRECT: 0.8803633, DIVERSITY_GINI: 0.0108182, SHANNON_ENTROPY: 7.1711550, RATIO_DIVERSITY_HERFINDAHL: 0.9849081, RATIO_DIVERSITY_GINI: 0.0404978, RATIO_SHANNON_ENTROPY: 0.5746229, RATIO_AVERAGE_POPULARITY: 3.0452625, RATIO_NOVELTY: 0.0290654, 

Iteration No: 14 ended. Evaluation done at random point.
Time taken: 17.7802
Function value obtained: -0.1053
Current

...

Iteration No: 170 ended. Search finished for the next optimal point.
Time taken: 19.1772
Function value obtained: -0.1180
Current minimum: -0.1180
Iteration No: 171 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.8090084838003266, 'beta': 0.28560307103110655}
MergeTwoModelsByScore: URM Detected 4 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13905 (100.0%) in 17.13 sec. Users per second: 812
SearchBayesianSkopt: Config 170 is suboptimal. Config: {'alpha': 0.8090084838003266, 'beta': 0.28560307103110655} - results: PRECISION: 0.2271845, PRECISION_RECALL_MIN_DEN: 0.2360085, RECALL: 0.0725542, MAP: 0.1099678, MAP_MIN_DEN: 0.1142328, MRR: 0.4479398, NDCG: 0.2381951, F1: 0.1099837, HIT_RATE: 0.9003236, ARHR_ALL_HITS: 0.6991499, NOVELTY: 0.0054990, AVERAGE_POPULARITY: 0.5891104, DIVERSITY_MEAN_INTER_LIST: 0.8487308, DIVERSITY_HERFINDAHL: 0.9848670, COVERAGE_ITEM: 0.1706628, COVERAGE_ITEM_CORRECT: 0.0864943, COVERAGE_USER: 0.99

Iteration No: 177 ended. Search finished for the next optimal point.
Time taken: 19.6000
Function value obtained: -0.1054
Current minimum: -0.1180
Iteration No: 178 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.3536859383669903, 'beta': 0.0}
MergeTwoModelsByScore: URM Detected 4 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13905 (100.0%) in 17.03 sec. Users per second: 816
SearchBayesianSkopt: Config 177 is suboptimal. Config: {'alpha': 0.3536859383669903, 'beta': 0.0} - results: PRECISION: 0.2436749, PRECISION_RECALL_MIN_DEN: 0.2487286, RECALL: 0.0712606, MAP: 0.1180249, MAP_MIN_DEN: 0.1202397, MRR: 0.4487741, NDCG: 0.2487878, F1: 0.1102728, HIT_RATE: 0.9107515, ARHR_ALL_HITS: 0.7253562, NOVELTY: 0.0056974, AVERAGE_POPULARITY: 0.4864356, DIVERSITY_MEAN_INTER_LIST: 0.9286488, DIVERSITY_HERFINDAHL: 0.9928582, COVERAGE_ITEM: 0.1719364, COVERAGE_ITEM_CORRECT: 0.0941913, COVERAGE_USER: 0.9943507, COVERAGE_USER_CORRECT: 0.

EvaluatorHoldout: Processed 13923 (100.0%) in 17.09 sec. Users per second: 815
SearchBayesianSkopt: Config evaluated with evaluator_test. Config: {'alpha': 0.970346419168093, 'beta': 4.40791884429358e-05} - results:
CUTOFF: 10 - PRECISION: 0.2851181, PRECISION_RECALL_MIN_DEN: 0.2893891, RECALL: 0.0713298, MAP: 0.1478912, MAP_MIN_DEN: 0.1496504, MRR: 0.5046840, NDCG: 0.2915537, F1: 0.1141116, HIT_RATE: 0.9385190, ARHR_ALL_HITS: 0.8576046, NOVELTY: 0.0056973, AVERAGE_POPULARITY: 0.4866304, DIVERSITY_MEAN_INTER_LIST: 0.9285064, DIVERSITY_HERFINDAHL: 0.9928440, COVERAGE_ITEM: 0.1727670, COVERAGE_ITEM_CORRECT: 0.1011684, COVERAGE_USER: 0.9956379, COVERAGE_USER_CORRECT: 0.9344251, DIVERSITY_GINI: 0.0191062, SHANNON_ENTROPY: 8.4588130, RATIO_DIVERSITY_HERFINDAHL: 0.9932105, RATIO_DIVERSITY_GINI: 0.0715240, RATIO_SHANNON_ENTROPY: 0.6778026, RATIO_AVERAGE_POPULARITY: 2.5010096, RATIO_NOVELTY: 0.0300881, 


SearchBayesianSkopt: Saving model in logs/MergeTwoModelsByScore

MergeTwoModelsByScore: S

Iteration No: 190 ended. Search finished for the next optimal point.
Time taken: 19.6562
Function value obtained: -0.1079
Current minimum: -0.1180
Iteration No: 191 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.48192841068610753, 'beta': 0.0}
MergeTwoModelsByScore: URM Detected 4 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13905 (100.0%) in 17.05 sec. Users per second: 815
SearchBayesianSkopt: Config 190 is suboptimal. Config: {'alpha': 0.48192841068610753, 'beta': 0.0} - results: PRECISION: 0.2436749, PRECISION_RECALL_MIN_DEN: 0.2487286, RECALL: 0.0712606, MAP: 0.1180249, MAP_MIN_DEN: 0.1202397, MRR: 0.4487741, NDCG: 0.2487878, F1: 0.1102728, HIT_RATE: 0.9107515, ARHR_ALL_HITS: 0.7253562, NOVELTY: 0.0056974, AVERAGE_POPULARITY: 0.4864356, DIVERSITY_MEAN_INTER_LIST: 0.9286488, DIVERSITY_HERFINDAHL: 0.9928582, COVERAGE_ITEM: 0.1719364, COVERAGE_ITEM_CORRECT: 0.0941913, COVERAGE_USER: 0.9943507, COVERAGE_USER_CORRECT: 

Iteration No: 197 ended. Search finished for the next optimal point.
Time taken: 19.8163
Function value obtained: -0.1180
Current minimum: -0.1180
Iteration No: 198 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.4364862406572628, 'beta': 0.0}
MergeTwoModelsByScore: URM Detected 4 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13905 (100.0%) in 17.03 sec. Users per second: 817
SearchBayesianSkopt: Config 197 is suboptimal. Config: {'alpha': 0.4364862406572628, 'beta': 0.0} - results: PRECISION: 0.2436749, PRECISION_RECALL_MIN_DEN: 0.2487286, RECALL: 0.0712606, MAP: 0.1180249, MAP_MIN_DEN: 0.1202397, MRR: 0.4487741, NDCG: 0.2487878, F1: 0.1102728, HIT_RATE: 0.9107515, ARHR_ALL_HITS: 0.7253562, NOVELTY: 0.0056974, AVERAGE_POPULARITY: 0.4864356, DIVERSITY_MEAN_INTER_LIST: 0.9286488, DIVERSITY_HERFINDAHL: 0.9928582, COVERAGE_ITEM: 0.1719364, COVERAGE_ITEM_CORRECT: 0.0941913, COVERAGE_USER: 0.9943507, COVERAGE_USER_CORRECT: 0.

In [13]:
#ITEMKNNCFCBF
ItemKNNCFCBF_recommender = ItemKNN_CFCBF_Hybrid_Recommender(URM_train=stacked_train, ICM_train=ICM)
ItemKNNCFCBF_recommender.fit(topK=252, ICM_weight=0.1,shrink=5.0,similarity="asymmetric")

ItemKNN_CFCBF_HybridRecommender: URM Detected 4 ( 0.0%) users with no interactions.
ItemKNN_CFCBF_HybridRecommender: ICM Detected 1 ( 0.0%) items with no features.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 18059 (100.0%), 1189.95 column/sec. Elapsed time 15.18 sec


In [16]:
similarity_h = similarity_hybrid.W_sparse
similarity_ItemCFCBF = ItemKNNCFCBF_recommender.W_sparse

In [17]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from skopt.space import Real
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

hyperparameters_range_dictionary = {
    "alpha": Real(0.0, 1.0)
}

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[stacked_train, similarity_h,similarity_ItemCFCBF ],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class=ItemKNNSimilarityHybridRecommender,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 200
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=ItemKNNSimilarityHybridRecommender.RECOMMENDER_NAME,
                    save_model="best"
                    )





Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'alpha': 0.19339005194365805}
ItemKNNSimilarityHybridRecommender: URM Detected 4 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13905 (100.0%) in 9.03 sec. Users per second: 1539
SearchBayesianSkopt: New best config found. Config 0: {'alpha': 0.19339005194365805} - results: PRECISION: 0.1997123, PRECISION_RECALL_MIN_DEN: 0.2035429, RECALL: 0.0569218, MAP: 0.0954350, MAP_MIN_DEN: 0.0973356, MRR: 0.4182929, NDCG: 0.2099003, F1: 0.0885930, HIT_RATE: 0.8512046, ARHR_ALL_HITS: 0.6290553, NOVELTY: 0.0054157, AVERAGE_POPULARITY: 0.6250043, DIVERSITY_MEAN_INTER_LIST: 0.8356827, DIVERSITY_HERFINDAHL: 0.9835623, COVERAGE_ITEM: 0.1678388, COVERAGE_ITEM_CORRECT: 0.0706573, COVERAGE_USER: 0.9943507, COVERAGE_USER_CORRECT: 0.8463959, DIVERSITY_GINI: 0.0084113, SHANNON_ENTROPY: 6.8777995, RATIO_DIVERSITY_HERFINDAHL: 0.9839253, RATIO_DIVERSITY_GINI: 0.0314876, RATIO_SHANNON_ENTROPY: 0.5

EvaluatorHoldout: Processed 13905 (100.0%) in 8.94 sec. Users per second: 1556
SearchBayesianSkopt: Config 4 is suboptimal. Config: {'alpha': 0.28196335531669664} - results: PRECISION: 0.2007767, PRECISION_RECALL_MIN_DEN: 0.2046841, RECALL: 0.0573769, MAP: 0.0960877, MAP_MIN_DEN: 0.0980018, MRR: 0.4211191, NDCG: 0.2111220, F1: 0.0892488, HIT_RATE: 0.8525710, ARHR_ALL_HITS: 0.6330493, NOVELTY: 0.0054154, AVERAGE_POPULARITY: 0.6249496, DIVERSITY_MEAN_INTER_LIST: 0.8357746, DIVERSITY_HERFINDAHL: 0.9835714, COVERAGE_ITEM: 0.1672296, COVERAGE_ITEM_CORRECT: 0.0707680, COVERAGE_USER: 0.9943507, COVERAGE_USER_CORRECT: 0.8477546, DIVERSITY_GINI: 0.0084137, SHANNON_ENTROPY: 6.8825509, RATIO_DIVERSITY_HERFINDAHL: 0.9839345, RATIO_DIVERSITY_GINI: 0.0314967, RATIO_SHANNON_ENTROPY: 0.5514971, RATIO_AVERAGE_POPULARITY: 3.2118938, RATIO_NOVELTY: 0.0285994, 

Iteration No: 5 ended. Evaluation done at random point.
Time taken: 9.8390
Function value obtained: -0.0961
Current minimum: -0.1095
Iteration No

EvaluatorHoldout: Processed 13905 (100.0%) in 9.05 sec. Users per second: 1537
SearchBayesianSkopt: Config 11 is suboptimal. Config: {'alpha': 0.17651169669361091} - results: PRECISION: 0.1996620, PRECISION_RECALL_MIN_DEN: 0.2034912, RECALL: 0.0568633, MAP: 0.0954445, MAP_MIN_DEN: 0.0973425, MRR: 0.4183685, NDCG: 0.2098779, F1: 0.0885171, HIT_RATE: 0.8508450, ARHR_ALL_HITS: 0.6291044, NOVELTY: 0.0054156, AVERAGE_POPULARITY: 0.6250478, DIVERSITY_MEAN_INTER_LIST: 0.8356509, DIVERSITY_HERFINDAHL: 0.9835591, COVERAGE_ITEM: 0.1678941, COVERAGE_ITEM_CORRECT: 0.0706019, COVERAGE_USER: 0.9943507, COVERAGE_USER_CORRECT: 0.8460383, DIVERSITY_GINI: 0.0084098, SHANNON_ENTROPY: 6.8766880, RATIO_DIVERSITY_HERFINDAHL: 0.9839222, RATIO_DIVERSITY_GINI: 0.0314820, RATIO_SHANNON_ENTROPY: 0.5510273, RATIO_AVERAGE_POPULARITY: 3.2123983, RATIO_NOVELTY: 0.0286005, 

Iteration No: 12 ended. Evaluation done at random point.
Time taken: 9.9664
Function value obtained: -0.0954
Current minimum: -0.1095
Iteration 

EvaluatorHoldout: Processed 13905 (100.0%) in 8.87 sec. Users per second: 1567
SearchBayesianSkopt: Config 18 is suboptimal. Config: {'alpha': 0.5217838856656382} - results: PRECISION: 0.2031787, PRECISION_RECALL_MIN_DEN: 0.2073965, RECALL: 0.0588420, MAP: 0.0975858, MAP_MIN_DEN: 0.0997215, MRR: 0.4241165, NDCG: 0.2139346, F1: 0.0912557, HIT_RATE: 0.8591873, ARHR_ALL_HITS: 0.6408023, NOVELTY: 0.0054157, AVERAGE_POPULARITY: 0.6242913, DIVERSITY_MEAN_INTER_LIST: 0.8363428, DIVERSITY_HERFINDAHL: 0.9836283, COVERAGE_ITEM: 0.1655130, COVERAGE_ITEM_CORRECT: 0.0705465, COVERAGE_USER: 0.9943507, COVERAGE_USER_CORRECT: 0.8543335, DIVERSITY_GINI: 0.0084281, SHANNON_ENTROPY: 6.8998363, RATIO_DIVERSITY_HERFINDAHL: 0.9839914, RATIO_DIVERSITY_GINI: 0.0315507, RATIO_SHANNON_ENTROPY: 0.5528822, RATIO_AVERAGE_POPULARITY: 3.2085105, RATIO_NOVELTY: 0.0286009, 

Iteration No: 19 ended. Evaluation done at random point.
Time taken: 9.7739
Function value obtained: -0.0976
Current minimum: -0.1095
Iteration N

EvaluatorHoldout: Processed 13905 (100.0%) in 8.99 sec. Users per second: 1547
SearchBayesianSkopt: Config 25 is suboptimal. Config: {'alpha': 0.1795518015501192} - results: PRECISION: 0.1997123, PRECISION_RECALL_MIN_DEN: 0.2035415, RECALL: 0.0568874, MAP: 0.0954635, MAP_MIN_DEN: 0.0973617, MRR: 0.4185115, NDCG: 0.2099291, F1: 0.0885513, HIT_RATE: 0.8512046, ARHR_ALL_HITS: 0.6292613, NOVELTY: 0.0054156, AVERAGE_POPULARITY: 0.6250301, DIVERSITY_MEAN_INTER_LIST: 0.8356658, DIVERSITY_HERFINDAHL: 0.9835606, COVERAGE_ITEM: 0.1679495, COVERAGE_ITEM_CORRECT: 0.0706019, COVERAGE_USER: 0.9943507, COVERAGE_USER_CORRECT: 0.8463959, DIVERSITY_GINI: 0.0084120, SHANNON_ENTROPY: 6.8770104, RATIO_DIVERSITY_HERFINDAHL: 0.9839237, RATIO_DIVERSITY_GINI: 0.0314901, RATIO_SHANNON_ENTROPY: 0.5510532, RATIO_AVERAGE_POPULARITY: 3.2123075, RATIO_NOVELTY: 0.0286006, 

Iteration No: 26 ended. Evaluation done at random point.
Time taken: 9.8701
Function value obtained: -0.0955
Current minimum: -0.1095
Iteration N

...

EvaluatorHoldout: Processed 13905 (100.0%) in 7.99 sec. Users per second: 1740
SearchBayesianSkopt: Config 174 is suboptimal. Config: {'alpha': 0.991047343480567} - results: PRECISION: 0.2335131, PRECISION_RECALL_MIN_DEN: 0.2385487, RECALL: 0.0687687, MAP: 0.1145970, MAP_MIN_DEN: 0.1171349, MRR: 0.4625986, NDCG: 0.2438864, F1: 0.1062478, HIT_RATE: 0.9047825, ARHR_ALL_HITS: 0.7249537, NOVELTY: 0.0057982, AVERAGE_POPULARITY: 0.4546717, DIVERSITY_MEAN_INTER_LIST: 0.9365980, DIVERSITY_HERFINDAHL: 0.9936531, COVERAGE_ITEM: 0.1962456, COVERAGE_ITEM_CORRECT: 0.1116895, COVERAGE_USER: 0.9943507, COVERAGE_USER_CORRECT: 0.8996711, DIVERSITY_GINI: 0.0239568, SHANNON_ENTROPY: 8.7400231, RATIO_DIVERSITY_HERFINDAHL: 0.9940199, RATIO_DIVERSITY_GINI: 0.0896821, RATIO_SHANNON_ENTROPY: 0.7003359, RATIO_AVERAGE_POPULARITY: 2.3367598, RATIO_NOVELTY: 0.0306209, 

Iteration No: 175 ended. Search finished for the next optimal point.
Time taken: 10.7593
Function value obtained: -0.1146
Current minimum: -0.114

/Users/gabriele/opt/anaconda3/envs/jupyter/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


EvaluatorHoldout: Processed 13905 (100.0%) in 7.98 sec. Users per second: 1742
SearchBayesianSkopt: Config 181 is suboptimal. Config: {'alpha': 0.9910269486305183} - results: PRECISION: 0.2335131, PRECISION_RECALL_MIN_DEN: 0.2385487, RECALL: 0.0687687, MAP: 0.1146053, MAP_MIN_DEN: 0.1171430, MRR: 0.4626743, NDCG: 0.2438955, F1: 0.1062479, HIT_RATE: 0.9047825, ARHR_ALL_HITS: 0.7250074, NOVELTY: 0.0057981, AVERAGE_POPULARITY: 0.4547047, DIVERSITY_MEAN_INTER_LIST: 0.9365817, DIVERSITY_HERFINDAHL: 0.9936514, COVERAGE_ITEM: 0.1961903, COVERAGE_ITEM_CORRECT: 0.1116895, COVERAGE_USER: 0.9943507, COVERAGE_USER_CORRECT: 0.8996711, DIVERSITY_GINI: 0.0239486, SHANNON_ENTROPY: 8.7395539, RATIO_DIVERSITY_HERFINDAHL: 0.9940182, RATIO_DIVERSITY_GINI: 0.0896513, RATIO_SHANNON_ENTROPY: 0.7002983, RATIO_AVERAGE_POPULARITY: 2.3369294, RATIO_NOVELTY: 0.0306205, 

Iteration No: 182 ended. Search finished for the next optimal point.
Time taken: 10.9011
Function value obtained: -0.1146
Current minimum: -0.11

EvaluatorHoldout: Processed 13905 (100.0%) in 7.98 sec. Users per second: 1743
SearchBayesianSkopt: Config 188 is suboptimal. Config: {'alpha': 0.9907746503040891} - results: PRECISION: 0.2335635, PRECISION_RECALL_MIN_DEN: 0.2386096, RECALL: 0.0687851, MAP: 0.1145867, MAP_MIN_DEN: 0.1171283, MRR: 0.4627153, NDCG: 0.2439045, F1: 0.1062726, HIT_RATE: 0.9045667, ARHR_ALL_HITS: 0.7249250, NOVELTY: 0.0057956, AVERAGE_POPULARITY: 0.4557750, DIVERSITY_MEAN_INTER_LIST: 0.9363519, DIVERSITY_HERFINDAHL: 0.9936285, COVERAGE_ITEM: 0.1959134, COVERAGE_ITEM_CORRECT: 0.1113018, COVERAGE_USER: 0.9943507, COVERAGE_USER_CORRECT: 0.8994565, DIVERSITY_GINI: 0.0238171, SHANNON_ENTROPY: 8.7303758, RATIO_DIVERSITY_HERFINDAHL: 0.9939953, RATIO_DIVERSITY_GINI: 0.0891591, RATIO_SHANNON_ENTROPY: 0.6995629, RATIO_AVERAGE_POPULARITY: 2.3424300, RATIO_NOVELTY: 0.0306071, 

Iteration No: 189 ended. Search finished for the next optimal point.
Time taken: 11.0037
Function value obtained: -0.1146
Current minimum: -0.11

EvaluatorHoldout: Processed 13905 (100.0%) in 7.99 sec. Users per second: 1739
SearchBayesianSkopt: Config 195 is suboptimal. Config: {'alpha': 0.9910254047303519} - results: PRECISION: 0.2335203, PRECISION_RECALL_MIN_DEN: 0.2385559, RECALL: 0.0687696, MAP: 0.1146069, MAP_MIN_DEN: 0.1171447, MRR: 0.4626803, NDCG: 0.2439000, F1: 0.1062496, HIT_RATE: 0.9047825, ARHR_ALL_HITS: 0.7250132, NOVELTY: 0.0057981, AVERAGE_POPULARITY: 0.4547022, DIVERSITY_MEAN_INTER_LIST: 0.9365817, DIVERSITY_HERFINDAHL: 0.9936514, COVERAGE_ITEM: 0.1961903, COVERAGE_ITEM_CORRECT: 0.1116895, COVERAGE_USER: 0.9943507, COVERAGE_USER_CORRECT: 0.8996711, DIVERSITY_GINI: 0.0239491, SHANNON_ENTROPY: 8.7395717, RATIO_DIVERSITY_HERFINDAHL: 0.9940182, RATIO_DIVERSITY_GINI: 0.0896532, RATIO_SHANNON_ENTROPY: 0.7002997, RATIO_AVERAGE_POPULARITY: 2.3369166, RATIO_NOVELTY: 0.0306206, 

Iteration No: 196 ended. Search finished for the next optimal point.
Time taken: 11.2196
Function value obtained: -0.1146
Current minimum: -0.11

In [18]:
recommenders = [similarity_hybrid, IALS_recommender]
new_hybrid = MergeTwoModelsByScores(stacked_train, recommenders)
new_hybrid.fit(alpha=0.4555360263375717,beta=0.13654581079541908)

MergeTwoModelsByScore: URM Detected 4 ( 0.0%) users with no interactions.


In [20]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from skopt.space import Real
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

hyperparameters_range_dictionary = {
    "alpha": Real(0.0, 1.0),
    "beta": Real(0.0, 1.0)
}
recommenders = [new_hybrid, ItemKNNCFCBF_recommender ]

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[stacked_train, recommenders],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class=MergeTwoModelsByScores,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 200
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=MergeTwoModelsByScores.RECOMMENDER_NAME,
                    save_model="best"
                    )





Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'alpha': 0.7096565015399446, 'beta': 0.4720294369946342}
MergeTwoModelsByScore: URM Detected 4 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13905 (100.0%) in 20.67 sec. Users per second: 673
SearchBayesianSkopt: New best config found. Config 0: {'alpha': 0.7096565015399446, 'beta': 0.4720294369946342} - results: PRECISION: 0.2027904, PRECISION_RECALL_MIN_DEN: 0.2066108, RECALL: 0.0580142, MAP: 0.0975270, MAP_MIN_DEN: 0.0994721, MRR: 0.4240945, NDCG: 0.2135186, F1: 0.0902186, HIT_RATE: 0.8572456, ARHR_ALL_HITS: 0.6406977, NOVELTY: 0.0053752, AVERAGE_POPULARITY: 0.6502515, DIVERSITY_MEAN_INTER_LIST: 0.7892596, DIVERSITY_HERFINDAHL: 0.9789203, COVERAGE_ITEM: 0.1547151, COVERAGE_ITEM_CORRECT: 0.0630710, COVERAGE_USER: 0.9943507, COVERAGE_USER_CORRECT: 0.8524027, DIVERSITY_GINI: 0.0065426, SHANNON_ENTROPY: 6.4254890, RATIO_DIVERSITY_HERFINDAHL: 0.9792817, RATIO_DIVERSITY_GI

EvaluatorHoldout: Processed 13905 (100.0%) in 20.35 sec. Users per second: 683
SearchBayesianSkopt: Config 6 is suboptimal. Config: {'alpha': 0.4402035838123317, 'beta': 0.7639607658015611} - results: PRECISION: 0.2000647, PRECISION_RECALL_MIN_DEN: 0.2037344, RECALL: 0.0568228, MAP: 0.0960741, MAP_MIN_DEN: 0.0979375, MRR: 0.4215055, NDCG: 0.2109002, F1: 0.0885075, HIT_RATE: 0.8528587, ARHR_ALL_HITS: 0.6338532, NOVELTY: 0.0053743, AVERAGE_POPULARITY: 0.6512421, DIVERSITY_MEAN_INTER_LIST: 0.7871089, DIVERSITY_HERFINDAHL: 0.9787052, COVERAGE_ITEM: 0.1555457, COVERAGE_ITEM_CORRECT: 0.0630157, COVERAGE_USER: 0.9943507, COVERAGE_USER_CORRECT: 0.8480406, DIVERSITY_GINI: 0.0065012, SHANNON_ENTROPY: 6.4012359, RATIO_DIVERSITY_HERFINDAHL: 0.9790665, RATIO_DIVERSITY_GINI: 0.0243373, RATIO_SHANNON_ENTROPY: 0.5129295, RATIO_AVERAGE_POPULARITY: 3.3470224, RATIO_NOVELTY: 0.0283826, 

Iteration No: 7 ended. Evaluation done at random point.
Time taken: 20.4882
Function value obtained: -0.0961
Current m

EvaluatorHoldout: Processed 13905 (100.0%) in 20.28 sec. Users per second: 686
SearchBayesianSkopt: Config 12 is suboptimal. Config: {'alpha': 0.31453454546078424, 'beta': 0.340560587801302} - results: PRECISION: 0.2012082, PRECISION_RECALL_MIN_DEN: 0.2048983, RECALL: 0.0572484, MAP: 0.0966425, MAP_MIN_DEN: 0.0985280, MRR: 0.4224026, NDCG: 0.2119421, F1: 0.0891356, HIT_RATE: 0.8545847, ARHR_ALL_HITS: 0.6364770, NOVELTY: 0.0053746, AVERAGE_POPULARITY: 0.6509017, DIVERSITY_MEAN_INTER_LIST: 0.7878534, DIVERSITY_HERFINDAHL: 0.9787797, COVERAGE_ITEM: 0.1550473, COVERAGE_ITEM_CORRECT: 0.0627942, COVERAGE_USER: 0.9943507, COVERAGE_USER_CORRECT: 0.8497569, DIVERSITY_GINI: 0.0065070, SHANNON_ENTROPY: 6.4096291, RATIO_DIVERSITY_HERFINDAHL: 0.9791410, RATIO_DIVERSITY_GINI: 0.0243588, RATIO_SHANNON_ENTROPY: 0.5136020, RATIO_AVERAGE_POPULARITY: 3.3452729, RATIO_NOVELTY: 0.0283842, 

Iteration No: 13 ended. Evaluation done at random point.
Time taken: 20.4209
Function value obtained: -0.0966
Current

EvaluatorHoldout: Processed 13905 (100.0%) in 20.36 sec. Users per second: 683
SearchBayesianSkopt: Config 18 is suboptimal. Config: {'alpha': 0.5254939781933562, 'beta': 0.791851950258564} - results: PRECISION: 0.2003164, PRECISION_RECALL_MIN_DEN: 0.2039851, RECALL: 0.0569044, MAP: 0.0962051, MAP_MIN_DEN: 0.0980788, MRR: 0.4217363, NDCG: 0.2111396, F1: 0.0886311, HIT_RATE: 0.8534340, ARHR_ALL_HITS: 0.6344630, NOVELTY: 0.0053744, AVERAGE_POPULARITY: 0.6511827, DIVERSITY_MEAN_INTER_LIST: 0.7872545, DIVERSITY_HERFINDAHL: 0.9787198, COVERAGE_ITEM: 0.1555457, COVERAGE_ITEM_CORRECT: 0.0630157, COVERAGE_USER: 0.9943507, COVERAGE_USER_CORRECT: 0.8486127, DIVERSITY_GINI: 0.0065060, SHANNON_ENTROPY: 6.4031068, RATIO_DIVERSITY_HERFINDAHL: 0.9790811, RATIO_DIVERSITY_GINI: 0.0243551, RATIO_SHANNON_ENTROPY: 0.5130794, RATIO_AVERAGE_POPULARITY: 3.3467174, RATIO_NOVELTY: 0.0283827, 

Iteration No: 19 ended. Evaluation done at random point.
Time taken: 20.4995
Function value obtained: -0.0962
Current 

EvaluatorHoldout: Processed 13905 (100.0%) in 21.19 sec. Users per second: 656
SearchBayesianSkopt: New best config found. Config 25: {'alpha': 0.7112081951498185, 'beta': 0.042450283307061805} - results: PRECISION: 0.2215750, PRECISION_RECALL_MIN_DEN: 0.2263578, RECALL: 0.0652481, MAP: 0.1062320, MAP_MIN_DEN: 0.1084288, MRR: 0.4314586, NDCG: 0.2293601, F1: 0.1008102, HIT_RATE: 0.8857246, ARHR_ALL_HITS: 0.6758167, NOVELTY: 0.0054058, AVERAGE_POPULARITY: 0.6276118, DIVERSITY_MEAN_INTER_LIST: 0.8262778, DIVERSITY_HERFINDAHL: 0.9826218, COVERAGE_ITEM: 0.1529431, COVERAGE_ITEM_CORRECT: 0.0680547, COVERAGE_USER: 0.9943507, COVERAGE_USER_CORRECT: 0.8807208, DIVERSITY_GINI: 0.0077955, SHANNON_ENTROPY: 6.8368761, RATIO_DIVERSITY_HERFINDAHL: 0.9829846, RATIO_DIVERSITY_GINI: 0.0291825, RATIO_SHANNON_ENTROPY: 0.5478372, RATIO_AVERAGE_POPULARITY: 3.2255760, RATIO_NOVELTY: 0.0285490, 

EvaluatorHoldout: Processed 13923 (100.0%) in 20.80 sec. Users per second: 669
SearchBayesianSkopt: Config evaluat

KeyboardInterrupt: 

In [ ]:
result_df, _ = evaluator_test.evaluateRecommender(new_hybrid)
print(result_df.loc[10])

In [ ]:
old_recommenders = [RP3Beta_recommender, IALS_recommender, SLIME_recommender]
old_hybrid = MergeThreeModelsByScores(stacked_train, old_recommenders)
old_hybrid.fit(alpha=0.0, beta=0.3023745211799622, gamma=1.0)

In [ ]:
result_df, _ = evaluator_test.evaluateRecommender(old_hybrid)
print(result_df.loc[10])

In [11]:
recommenders = [similarity_hybrid, IALS_recommender]
hybrid = MergeTwoModelsByScores(URM_stacked, recommenders)
hybrid.fit(alpha=0.4555360263375717,beta=0.13654581079541908)

In [12]:
submission = create_submission(hybrid)

100%|████████████████████████████████████▉| 18056/18059 [21:42<00:00, 13.86it/s]


In [13]:
write_submission(submission, 'RP3hybridSLIM_IALS_stacked')